In [1]:
import nltk
from nltk import NaiveBayesClassifier as nbc
from nltk.classify import accuracy
from pythainlp.tokenize import word_tokenize
import codecs
from itertools import chain
import pickle
#from pythainlp.transliterate import romanize
#from pythainlp.translate import download_model_all
from pythainlp.summarize import summarize
import pandas as pd
import time
#from pandarallel import pandarallel
from math import sin

In [2]:
print("Start Program")
f = open('finalized_model.pickle', 'rb')
classifier = pickle.load(f)
f.close()

f = open('vocabulary_model.pickle', 'rb')
vocabulary = pickle.load(f)
f.close()

Start Program


In [3]:
print("Run Predict")
x=0
def mypredit(fname):
    global x
    x =x+1
    print(x)
    test_sentence = fname
    featurized_test_sentence={i:(i in word_tokenize(test_sentence.lower()))for i in vocabulary}
    return classifier.classify(featurized_test_sentence)

df = pd.read_csv('28_08_2023 17_35_12.csv')
df.head()
df['Result'] = df['text'].apply(mypredit)
df.to_csv('28_08_2023 17_35_12P.csv')
print("ok")

Run Predict
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
ok


In [19]:
import json, pickle

PICKLE_PATH = "vocabulary_model.pickle"
JSON_PATH   = "vocabulary_words.json"

with open(PICKLE_PATH, "rb") as f:
    vocab = pickle.load(f)

# แปลงเป็น list ถ้ายังเป็น set
if isinstance(vocab, set):
    vocab = sorted(list(vocab))

with open(JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)

print("✅ Saved safe JSON vocabulary:", JSON_PATH)


✅ Saved safe JSON vocabulary: vocabulary_words.json


In [21]:
import json, math
import pandas as pd
from pythainlp.tokenize import word_tokenize

MODEL_JSON = "finalized_model_nltk_nb.json"
VOCAB_JSON = "vocabulary_words.json"
INPUT_CSV  = "28_08_2023 17_35_12.csv"
OUTPUT_CSV = "28_08_2023 17_35_12P.csv"

# ---------- โหลดโมเดล NB ที่แปลงเป็น JSON ----------
def load_safe_nb(json_path: str) -> dict:
    with open(json_path, "r", encoding="utf-8") as f:
        M = json.load(f)
    assert M.get("model_type") == "nltk_naive_bayes", "ไฟล์โมเดลไม่ใช่ฟอร์แมต NLTK NB"
    # บังคับชนิด float
    M["label_priors"] = {k: float(v) for k, v in M["label_priors"].items()}
    for lbl, d in M["feature_probs"].items():
        for fname, bucket in d.items():
            for val, p in list(bucket.items()):
                bucket[val] = float(p)
    M["smoothing_epsilon"] = float(M.get("smoothing_epsilon", 1e-12))
    return M

def predict_proba_safe_nb(M: dict, features: dict) -> dict:
    labels = M["labels"]
    priors = M["label_priors"]
    fprobs = M["feature_probs"]
    eps = float(M["smoothing_epsilon"])

    logps = []
    for lbl in labels:
        lp = math.log(max(priors.get(lbl, eps), eps))
        # ตรงตามโค้ดเก่า: ใส่ทุก feature ใน vocabulary เป็น True/False
        for fname, fval in features.items():
            bucket = fprobs.get(lbl, {}).get(fname, {})
            p = float(bucket.get(str(fval), eps))  # คีย์เป็นสตริง "True"/"False"
            lp += math.log(max(p, eps))
        logps.append(lp)

    # softmax
    m = max(logps)
    exps = [math.exp(lp - m) for lp in logps]
    Z = sum(exps) or 1.0
    probs = [e / Z for e in exps]
    return dict(zip(labels, probs))

def predict_safe_nb(M: dict, features: dict):
    probs = predict_proba_safe_nb(M, features)
    return max(probs.items(), key=lambda kv: kv[1])[0], probs

# ---------- โหลด vocabulary ----------
def load_vocabulary(path: str):
    with open(path, "r", encoding="utf-8") as f:
        vocab = json.load(f)
    # เผื่อไฟล์เป็น set/list ปนกัน ให้เป็น list ธรรมดา
    return list(vocab)

# ---------- แปลงข้อความ -> ฟีเจอร์ (เหมือนโค้ดเก่า) ----------
def featurize(text: str, vocabulary) -> dict:
    tokens = set(word_tokenize(str(text).lower()))
    # เหมือนเดิมเป๊ะ: {i: (i in tokens) for i in vocabulary}
    return {w: (w in tokens) for w in vocabulary}

# ---------- main ----------
def main():
    print("Start Program (safe JSON)")
    M = load_safe_nb(MODEL_JSON)
    vocabulary = load_vocabulary(VOCAB_JSON)
    print("Run Predict")

    df = pd.read_csv(INPUT_CSV)
    # สร้างคอลัมน์ผลลัพธ์
    cnt = 0
    def _predict_one(txt):
        nonlocal cnt
        cnt += 1
        if cnt % 100 == 0:
            print(cnt)
        feats = featurize(txt, vocabulary)
        yhat, _ = predict_safe_nb(M, feats)
        return yhat

    df["Result"] = df["text"].apply(_predict_one)
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print("ok ->", OUTPUT_CSV)

if __name__ == "__main__":
    main()

Start Program (safe JSON)
Run Predict
100
ok -> 28_08_2023 17_35_12P.csv
